CNCF: Estrategia caida normal caida fuerte
1. Los precios caigan y toquen PM de 40, si los precios caen de 3-5 puntos es una Caida Normal, Si los precios tocan el PM de 40 y siguen cayendo de 6 o mas puntos es una Caida fuerte
2. Si se da una caida de 6 o mas y luego salen 2 velas martillo lo cual me indica que esta parando de caer
3. Trazar linea trend siguendo la caida
4. Si rompe con vela verde alcista, vela final despues de las 11:00am Buy un CALL,  los precios pueden subir durante ese dia y el dia siguente.
5. Vende cuando has alcanzado rentabilidad, con el limit, o en primera vela roja de apertura

In [2]:
import pandas as pd
import numpy as np
from datetime import date, datetime
from scipy.signal import argrelextrema
from statsmodels.nonparametric.kernel_regression import KernelReg
from scipy import stats

In [4]:
tickers = [
'SPY',
'AAPL'
]

In [6]:
file_path = r'D:\traderxpro\data\df_h.txt'
#df = pd.read_csv(file_path, sep='\t', dtype={'open':float, 'high':float, 'low':float, 'close':float, 'volume':float,'fecha':datetime, 'companyName':str, 'fecha2:':datetime})
df = pd.read_csv(file_path, sep='\t')
df['fecha']=pd.to_datetime(df['fecha'])
df['fecha2']=pd.to_datetime(df['fecha2'])

In [8]:
df.query("fecha2>'2024-07-29' and fecha2<'2024-07-31'")

,fecha,open,high,low,close,volume,companyName,fecha2
1054,2024-07-30 20:00:00,542.00,542.0000,542.000,542.00,1065627,SPY,2024-07-30 21:00:00
1055,2024-07-30 19:00:00,541.43,542.8500,541.310,542.74,254331,SPY,2024-07-30 20:00:00
1056,2024-07-30 18:00:00,540.10,542.5100,539.845,541.42,1288025,SPY,2024-07-30 19:00:00
1057,2024-07-30 17:00:00,539.90,542.0000,539.000,540.00,212689,SPY,2024-07-30 18:00:00
1058,2024-07-30 16:00:00,541.92,542.5514,533.010,539.90,8542429,SPY,2024-07-30 17:00:00
...,...,...,...,...,...,...,...,...
9352,2024-07-29 08:00:00,217.75,219.3500,216.620,217.03,314522,AAPL,2024-07-29 09:00:00
9353,2024-07-29 07:00:00,217.77,218.2300,217.500,218.09,78800,AAPL,2024-07-29 08:00:00
9354,2024-07-29 06:00:00,218.65,218.7000,216.810,217.72,59428,AAPL,2024-07-29 07:00:00
9355,2024-07-29 05:00:00,218.28,218.8200,218.220,218.69,16694,AAPL,2024-07-29 06:00:00


In [10]:
df['hour'] = df['fecha2'].dt.hour

In [12]:
dfprev=df.query("hour>=9 and hour<=16").sort_values(by=["fecha2"],ascending=True)

In [14]:
dfprev=dfprev.reset_index()
dfprev=dfprev.drop(['index'], axis=1)

In [16]:
dfprev=dfprev.drop(['hour', 'fecha'], axis=1)
#renombrar fecha2 to fecha
dfprev=dfprev.rename(columns={"fecha2": "datetime"})

In [18]:
# 1. Los precios caigan y toquen PM de 40, si los precios caen de 3-5 puntos es una Caida Normal, Si los precios tocan el PM de 40 y 
# siguen cayendo de 6 o mas puntos es una Caida fuerte
df2 = pd.DataFrame()
for ticker in tickers:
    dataTicker = dfprev.query("companyName==@ticker")
    dataTicker=dataTicker.reset_index()
    dataTicker=dataTicker.drop(['index'], axis=1)
    dataTicker['SMA20'] = dataTicker['close'].rolling(20).mean()
    dataTicker['SMA40'] = dataTicker['close'].rolling(40).mean()
    dataTicker['SMA100'] = dataTicker['close'].rolling(100).mean()
    dataTicker['SMA200'] = dataTicker['close'].rolling(200).mean()
    if (df2.shape[0]<=0):
        df2 = dataTicker
    else:
        df2 = pd.concat([df2, dataTicker], ignore_index=True)

In [20]:
df2.query("companyName=='AAPL'").loc[3992:4020]

,open,high,low,close,volume,companyName,datetime,SMA20,SMA40,SMA100,SMA200
3992,154.7900,155.6500,154.0000,155.5700,323707,AAPL,2022-11-01 09:00:00,NaN,NaN,NaN,NaN
3993,155.5700,155.6501,152.6800,152.8630,9304836,AAPL,2022-11-01 10:00:00,NaN,NaN,NaN,NaN
3994,152.8700,152.9900,149.6600,150.0500,16860249,AAPL,2022-11-01 11:00:00,NaN,NaN,NaN,NaN
3995,150.0500,150.3100,149.1300,150.0600,8907564,AAPL,2022-11-01 12:00:00,NaN,NaN,NaN,NaN
3996,150.0500,150.5000,149.6100,150.0650,6804919,AAPL,2022-11-01 13:00:00,NaN,NaN,NaN,NaN
3997,150.0600,150.3700,149.6150,150.1100,4591602,AAPL,2022-11-01 14:00:00,NaN,NaN,NaN,NaN
3998,150.1050,151.0900,149.9600,150.9700,6416335,AAPL,2022-11-01 15:00:00,NaN,NaN,NaN,NaN
3999,150.9600,151.0500,150.4210,150.5700,10270283,AAPL,2022-11-01 16:00:00,NaN,NaN,NaN,NaN
4000,149.9000,150.0000,148.4500,148.8600,365678,AAPL,2022-11-02 09:00:00,NaN,NaN,NaN,NaN
4001,148.8600,150.6000,148.4400,148.6800,7903546,AAPL,2022-11-02 10:00:00,NaN,NaN,NaN,NaN


In [22]:
df2.query("(SMA40-close)>=3")

,open,high,low,close,volume,companyName,datetime,SMA20,SMA40,SMA100,SMA200
96,393.4700,397.1200,390.2700,390.3600,1355548,SPY,2022-11-17 09:00:00,397.634935,396.418855,NaN,NaN
97,390.3600,391.7000,390.0400,390.3744,6843364,SPY,2022-11-17 10:00:00,397.148650,396.433715,NaN,NaN
98,390.3300,392.2000,390.1400,390.5600,7948516,SPY,2022-11-17 11:00:00,396.739150,396.436840,NaN,NaN
99,390.5410,393.1800,390.2900,393.1400,6609381,SPY,2022-11-17 12:00:00,396.637150,396.468340,386.222327,NaN
102,394.2200,394.6500,392.5000,393.1500,5487537,SPY,2022-11-17 15:00:00,395.502205,396.693118,386.414531,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7971,229.2500,230.2000,228.8806,229.5800,3115136,AAPL,2024-10-24 12:00:00,232.495055,233.722508,231.220116,229.182355
7972,229.5900,230.3450,228.6700,230.3080,2467514,AAPL,2024-10-24 13:00:00,232.295455,233.675805,231.288792,229.187995
7973,230.3100,230.8200,230.1100,230.2550,2339177,AAPL,2024-10-24 14:00:00,232.150205,233.621430,231.340642,229.192371
7974,230.2600,230.5676,230.0400,230.4600,1640595,AAPL,2024-10-24 15:00:00,231.961955,233.580430,231.394842,229.198071


In [24]:
## Funcion minimos y maximos por HORA

def find_extrema_h(s, col, bw='cv_ls'):
    """
    Input:
        s: prices as pd.series
        bw: bandwith as str or array like
    Returns:
        prices: with 0-based index as pd.series
        extrema: extrema of prices as pd.series
        smoothed_prices: smoothed prices using kernel regression as pd.series
        smoothed_extrema: extrema of smoothed_prices as pd.series
    """
    # Copy series so we can replace index and perform non-parametric
    # kernel regression.
    prices = s.copy()
    prices = prices.reset_index()
    
    #print(prices.head(5))
    prices.columns = ['index','datetime',col]
    
    #prices.columns = ['date', 'close', 'date2']
    horas = prices['datetime']
    prices = prices[col]    
    
    #prices = s.copy()
    #prices.columns = ['date', 'price']
    #prices = pd.DataFrame(s)
    #prices = prices['close']
    
    #print(type(prices))
    #print("===============")
    #print(prices)
    #print("===============")
    #print(prices.index)

    kr = KernelReg(
        [prices.values],
        [prices.index.to_numpy()],
        var_type='c', bw=bw
    )
    f = kr.fit([prices.index.values])

    # Use smoothed prices to determine local minima and maxima
    smooth_prices = pd.Series(data=f[0], index=prices.index)
    smooth_local_max = argrelextrema(smooth_prices.values, np.greater)[0]
    smooth_local_min = argrelextrema(smooth_prices.values, np.less)[0]
    local_max_min = np.sort(
        np.concatenate([smooth_local_max, smooth_local_min]))
    smooth_extrema = smooth_prices.loc[local_max_min]

    # Iterate over extrema arrays returning datetime of passed
    # prices array. Uses idxmax and idxmin to window for local extrema.
    price_local_max_dt = []
    for i in smooth_local_max:
        if (i > 1) and (i < len(prices)-1):
            price_local_max_dt.append(prices.iloc[i-2:i+2].idxmax())

    price_local_min_dt = []
    for i in smooth_local_min:
        if (i > 1) and (i < len(prices)-1):
            price_local_min_dt.append(prices.iloc[i-2:i+2].idxmin())

    maxima = pd.Series(prices.loc[price_local_max_dt])
    minima = pd.Series(prices.loc[price_local_min_dt])
    extrema = pd.concat([maxima, minima]).sort_index()
  

    # Return series for each with bar as index
    return extrema, prices, smooth_extrema, smooth_prices, maxima, minima, horas

In [26]:
# Determinar puntos minimos por HORA
df_h2 = df2
index = 1
df_min = pd.DataFrame()
df_max = pd.DataFrame()
for ticker in tickers:
    print(ticker)
    print(index)
    index+=1
    tuplas = df_h2.query("companyName==@ticker")
    #col1 = 'close'
    
    colmin='close'
    tuplas2 = tuplas[['datetime',colmin]]   
    extrema, prices, smooth_extrema, smooth_prices, maxima, minima, horas = find_extrema_h(tuplas2, colmin, bw=[1.5])
    
    minima2 = minima.to_frame()
    minima2 = minima2.drop_duplicates()
    minima2.rename(columns={"close": "trendlower"}, inplace = True)
    minima2['companyName']=ticker    
    #minima2 = minima2.reset_index()
    minima2 = pd.concat([minima2, horas], join = 'outer', axis=1)
    
    colmax='close'
    tuplas3 = tuplas[['datetime',colmax]]
    extrema, prices, smooth_extrema, smooth_prices, maxima, minima, horas = find_extrema_h(tuplas3, colmax, bw=[1.5])

    maxima2 = maxima.to_frame()
    maxima2 = maxima2.drop_duplicates()
    maxima2.rename(columns={"close": "trendhigher"}, inplace = True)
    maxima2['companyName']=ticker
    #maxima2 = maxima2.reset_index()
    maxima2 = pd.concat([maxima2, horas], join = 'outer', axis=1)
    
    if index==1:        
        df_min = minima2
        df_max = maxima2
    else:
        df_min = pd.concat([df_min, minima2],ignore_index=True)
        df_max = pd.concat([df_max, maxima2],ignore_index=True)
    #del tuplas, tuplas2, extrema, prices, smooth_extrema, smooth_prices, maxima, minima, minima2, maxima2


#print(df_min.head(5))
#print(df_max.head(5))
df_h2 = pd.merge(df_h2, df_min[['datetime','companyName', 'trendlower']], on = ['companyName','datetime'], how='left')
df_h2 = pd.merge(df_h2, df_max[['datetime','companyName', 'trendhigher']], on = ['companyName','datetime'], how='left')

SPY
1
AAPL
2


In [28]:
#funcion para detectar vela martillo hombre colgado y martillo invertido
def vela_martillo(df_h2):
    tipo=""
    if  (
        ((df_h2["high"] - df_h2["low"]) > 3 * (df_h2["open"] - df_h2["close"])) 
        & (((df_h2["close"] - df_h2["low"]) / (0.001 + df_h2["high"] - df_h2["low"])) > 0.6) 
        & (((df_h2["open"] - df_h2["low"]) / (0.001 + df_h2["high"] - df_h2["low"])) > 0.6)):
        tipo="hammer"    
    elif (
        ((df_h2["high"] - df_h2["low"]) > 3 * (df_h2["open"] - df_h2["close"]))
        & ((df_h2["high"] - df_h2["close"]) / (0.001 + df_h2["high"] - df_h2["low"]) > 0.6)
        & ((df_h2["high"] - df_h2["open"]) / (0.001 + df_h2["high"] - df_h2["low"]) > 0.6)):
        tipo="inverted_hammer"
    else:
        tipo=""
    return tipo

In [30]:
df_h2["tip_vela"] = df_h2.apply(vela_martillo, axis=1)

In [32]:
df_h2[(df_h2["tip_vela"]=="inverted_hammer") & (df_h2["companyName"]=="AAPL")]

,open,high,low,close,volume,companyName,datetime,SMA20,SMA40,SMA100,SMA200,trendlower,trendhigher,tip_vela
4001,148.8600,150.6000,148.4400,148.6800,7903546,AAPL,2022-11-02 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,inverted_hammer
4006,149.0400,152.1700,147.3500,147.7850,19890348,AAPL,2022-11-02 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,inverted_hammer
4020,135.0400,136.5599,134.4900,134.8004,14933460,AAPL,2022-11-04 13:00:00,142.398420,NaN,NaN,NaN,134.8004,NaN,inverted_hammer
4021,134.8100,135.6500,134.4500,134.8800,8705089,AAPL,2022-11-04 14:00:00,141.708420,NaN,NaN,NaN,NaN,NaN,inverted_hammer
4024,136.3102,138.6500,135.4515,136.4000,1123421,AAPL,2022-11-07 09:00:00,139.914690,NaN,NaN,NaN,NaN,NaN,inverted_hammer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7898,227.9650,229.1900,227.6200,228.0900,4177446,AAPL,2024-10-11 11:00:00,228.091885,226.385163,226.810078,224.607729,NaN,NaN,inverted_hammer
7923,230.4000,231.1450,230.1102,230.1988,3375293,AAPL,2024-10-16 12:00:00,232.161715,230.283915,227.860380,226.056316,NaN,NaN,inverted_hammer
7960,234.5500,235.8600,233.7500,233.9400,139448,AAPL,2024-10-23 09:00:00,235.252345,233.897662,230.530681,228.748517,NaN,NaN,inverted_hammer
7968,230.9637,234.1750,229.7700,230.4983,164236,AAPL,2024-10-24 09:00:00,233.524305,233.896910,231.012714,229.157156,NaN,NaN,inverted_hammer


In [34]:
#puntos de cambio de tendencia alcista, caida normal y caida fuerte
def cncf01(df_h2):
    if ((df_h2['trendlower']>0) & ((df_h2['SMA40']-df_h2['close'])>=3)):
        return 1
    else:
        return 0

def tip_caida(df_h2):
    if ((df_h2['trendlower']>0) & (((df_h2['SMA40']-df_h2['close'])>=3) & ((df_h2['SMA40']-df_h2['close'])<6))):
        return 'CN'
    elif ((df_h2['trendlower']>0) & ((df_h2['SMA40']-df_h2['close'])>=6)):
        return 'CF'

df_h2['cncf01'] = df_h2.apply(cncf01, axis=1)
df_h2['tip_caida'] = df_h2.apply(tip_caida, axis=1)

In [ ]:
df_h2[(df_h2["cncf01"]==1)  & (df_h2["companyName"]=="SPY") & (df_h2["datetime"]>="2023-10-02 00:00:00")]

In [ ]:
#numeracion de casos
df_h2['id_cncf01'] = df_h2.query("cncf01==1").groupby(["companyName"]).cumcount()+1

In [ ]:
df_h2['id_cncf01'].drop_duplicates()

In [ ]:
#obtener 20 datos anteriores y despues, dataframe HORA
df_h3 = pd.DataFrame()
posiciones = 20 #horas que entran en evaluacion por caso
#i=4046
for i, row in df_h2.iterrows():
    #print("index:" ,i, "company:", df_h2['companyName'][i])
    company = df_h2['companyName'][i]
    if (df_h2['cncf01'][i]==1):
        print("index insertar:",i, "company:", company)
        dataframe = df_h2.query("companyName==@company")
        tupla = dataframe.query("index==@i and companyName==@company")
        
        #tupla = df_h2.loc[i]
        tupla = tupla.reset_index(drop=True)
        tupla['ind_posicion']=0 #punto central
        
        #print("id_cncf:", tupla["id_cncf01"])
        #tupla = df_d2.loc[i]
        if len(df_h3)<1:
            df_h3 = tupla
        else:
            df_h3 = pd.concat([df_h3, tupla],ignore_index=True)
        
        id_cncf = dataframe.loc[i, 'id_cncf01']
        print ("id_cncf:", id_cncf)
        
        #usamos funcion loc
        filas_posteriores = dataframe.loc[i+1: i + posiciones ]
        filas_posteriores = filas_posteriores.reset_index(drop=True)
        filas_posteriores['cncf01']=0
        filas_posteriores['id_cncf01']=id_cncf
        filas_posteriores['ind_posicion']=1 #registros posteriores
        
        filas_anteriores = dataframe.loc[max(0, i - posiciones):i-1]
        filas_anteriores = filas_anteriores.reset_index(drop=True)
        filas_anteriores['cncf01']=0
        filas_anteriores['id_cncf01']=id_cncf
        filas_anteriores['ind_posicion']=-1 #registros anteriores
        
        df_h3 = pd.concat([df_h3, filas_anteriores],ignore_index=True)
        df_h3 = pd.concat([df_h3, filas_posteriores],ignore_index=True)

In [ ]:
#df_h2.query("id_cncf01==110")
#df_h3
#dataframe
#filas_posteriores
#dataframe.iloc[4046 +1: 4046  + (20 + 1)]

#df_h2.query("companyName=='AAPL'").loc[4044:4099]

In [ ]:
df_h3=df_h3.sort_values(by=["companyName","id_cncf01","datetime"])
df_h3=df_h3.reset_index()
df_h3=df_h3.drop(['index'], axis=1)

In [ ]:
#df_h3.query("index>=107 and index<=112 and tip_vela=='inverted_hammer' and companyName=='AAPL'").shape[0]
#df_h3.query("id_cncf01>0").shape[0]/41
#df_h3.groupby(by=["companyName","id_cncf01"]).count()

#df_h3.query("id_cncf01==111 and companyName=='SPY'").shape[0]

df_h3.query("id_cncf01==110 and companyName=='SPY'").shape[0]

In [ ]:
#Hallando martillo invertido, indicador que se esta parando la caida
#obteniendo casos
df_casos = df_h3[['companyName','id_cncf01']].drop_duplicates()
df_casos2 = pd.DataFrame()
for i, row in df_casos.iterrows():
    #print("item:", i)
    print("companyName:",df_casos['companyName'][i],"id_cncf01:",df_casos["id_cncf01"][i])
    company=df_casos['companyName'][i]
    id_cncf01=df_casos['id_cncf01'][i]
    tupla1=df_h3[["companyName","id_cncf01","ind_posicion"]].query("companyName==@company and id_cncf01==@id_cncf01 and ind_posicion==0")
    id=tupla1.index
    print("index central:", tupla1.index)
    #revisamos 3 velas arriba 2 velas abajo, lo ideal es que se encuentre 2 velas martillo invertido consecutivas
    ini = id-3
    fin = id+2
    conteo=1
    cnt_invertedHammer=0
    while conteo<=6:
        fin = ini + 1
        print("ini:",ini,"fin:",fin, "company", company)
        casos = df_h3.query("index>=@ini and index<=@fin and tip_vela=='inverted_hammer' and companyName==@company").shape[0]
        if casos>=2:
            cnt_invertedHammer += 1
        ini = fin
        conteo+=1
    if (cnt_invertedHammer>=1):
        if len(df_casos2)<1:
            print("insert 1")
            df_casos2 = tupla1
        else:
            print("insert 2")
            df_casos2 = pd.concat([df_casos2, tupla1],ignore_index=True)

In [ ]:
#numeracion de casos previo al final
df_casos2['id_cncf'] = df_casos2.groupby(["companyName"]).cumcount()+1

In [ ]:
#borrando columna que no se necesita
df_casos2=df_casos2.drop('ind_posicion',axis=1)

In [ ]:
df_h4 = pd.merge(df_h3,df_casos2, how="inner",on=["companyName","id_cncf01"])

In [ ]:
#crea la linea trend solo con los 2 valores HIGH mas altos a partir del ultimo trendlower
def collect_channel(candle, backcandles, window, id_caso):
    #localdf = df_h4[candle-backcandles-window:candle-window]
    
    #highs = dfpl[(dfpl['id_cncf']==id_caso) & (dfpl['ind_high']>0) & (dfpl.index<=candle)].high.values
    #idxhighs = dfpl[(dfpl['id_cncf']==id_caso) & (dfpl['ind_high']>0) & (dfpl.index<=candle)].high.index
    highs = dfpl[(dfpl['id_cncf']==id_caso)  & (dfpl.index<=candle)].high.values
    idxhighs = dfpl[(dfpl['id_cncf']==id_caso) & (dfpl.index<=candle)].high.index

    if len(highs)>=2:
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)

In [ ]:
#crea la linea trend con todos los datos HIGH a partir del ultimo trendlower
def collect_channel2(candle, backcandles, window, id_caso):
    #localdf = df_h4[candle-backcandles-window:candle-window]
    tupla = dfpl[['datetime', 'id_cncf', 'iniTrendLow']].query("id_cncf==@id_caso and iniTrendLow==True")
    fec = tupla['datetime'].iloc[0]
    print("fechaaaaaaa:", fec)
    
    highs = dfpl[(dfpl['id_cncf']==id_caso) & (dfpl['datetime']>=fec) & (dfpl.index<=candle)].high.values
    idxhighs = dfpl[(dfpl['id_cncf']==id_caso) & (dfpl['datetime']>=fec) & (dfpl.index<=candle)].high.index

    if len(highs)>=2:
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)

In [ ]:
#puntos mas altos para hacer la linea de tendencia
#casos
top_valuesxcasos = pd.DataFrame()
for i, row in df_casos2.iterrows():
    id_cncf = df_casos2['id_cncf'][i]
    company = df_casos2['companyName'][i]    
    #id_cncf = 6
    #company = 'AAPL'
    print("id_cncf:" ,id_cncf, "company:", company)
    #ultimo cambio de tendencia a la baja
    tupla1 = df_h4[['datetime','companyName','id_cncf','ind_posicion','trendhigher']].query("companyName==@company and id_cncf==@id_cncf and ind_posicion<0 and trendhigher>0").nlargest(1, 'datetime')
    if (tupla1.size>0):
        # si existe el cambio de tendencia a la baja se busca el high mayor desde este ultimo cambio de tendencia
        fec1 = tupla1['datetime'].iloc[0]
        print("fec1",fec1)
        tupla = df_h4[['datetime','companyName','id_cncf','ind_posicion','high']].query("companyName==@company and id_cncf==@id_cncf and ind_posicion<0 and datetime>=@fec1").nlargest(1, 'high')
                
    else:
        # si no existe el cambio de tendencia a la baja que evalue todo lo de atras (datetime menor)
        print("op2")
        tupla = df_h4[['datetime','companyName','id_cncf','ind_posicion','high']].query("companyName==@company and id_cncf==@id_cncf and ind_posicion<0").nlargest(1, 'high')
    
    fec = tupla['datetime'].iloc[0]
    print("fec",fec)
    #Obtener 2 tuplas con mayor high a partir del ultimo trendlower
    tuplas = df_h4.query("id_cncf==@id_cncf and companyName==@company and datetime>=@fec and ind_posicion<=0").nlargest(3, 'high')
    tuplas['iniTrendLow'] =  np.where(tuplas['datetime']==tuplas['datetime'].min(), True, False) #fecha donde inicia el primer trendlower
    top_valuesxcasos = pd.concat([top_valuesxcasos, tuplas],ignore_index=True)

In [ ]:
top_valuesxcasos['ind_high'] = top_valuesxcasos['high']

In [ ]:
df_h5 = pd.merge(df_h4, top_valuesxcasos[['datetime','companyName', 'id_cncf', 'ind_high', 'iniTrendLow']], on = ['companyName', 'id_cncf','datetime'], how='left')

In [ ]:
df_h5["x"] = df_h5.index

In [ ]:
#Hacemos nuevamente el recorrido de los casos finales para obtener el trendlower
dfTrendTotal = pd.DataFrame()
dfpl = pd.DataFrame()
#company='AAPL'
#id_cncf=6
for i, row in df_casos2.iterrows():
    company=df_casos2['companyName'][i]
    id_cncf=df_casos2['id_cncf'][i]
    print("companyName:",company,"id_cncf:",id_cncf)
    dfpl=df_h5.query("companyName==@company and id_cncf==@id_cncf")
    tupla = dfpl.query("ind_posicion==0")
    candle = tupla.index[0]
    backcandles = 15
    window = 0
    #id_caso = df_casos2['id_cncf'][i]
    sl_highs, interc_highs,  r_sq_h = collect_channel2(candle, backcandles, window, id_cncf)

    #Si por los datos no es una linea trend a la baja, evaluar todas la velas antes del caso
    if interc_highs<=0:
        sl_highs, interc_highs,  r_sq_h = collect_channel(candle, backcandles, window, id_cncf)
    
    x = np.array(range(candle-backcandles-window, candle+5))
    print("sl_highs:",sl_highs,"interc_highs",interc_highs)    
    dfTrend = pd.DataFrame(x, columns=["x"])
    dfTrend["companyName"] = company
    dfTrend["id_cncf"] = id_cncf
    dfTrend["sl_highs"] = sl_highs
    dfTrend["interc_highs"] = interc_highs
    dfTrend['trend'] = sl_highs*dfTrend['x'] + interc_highs
        
    if len(dfTrendTotal)<1:
        dfTrendTotal = dfTrend
    else:
        dfTrendTotal = pd.concat([dfTrendTotal, dfTrend],ignore_index=True)

In [ ]:
df_h6 = pd.merge(df_h5, dfTrendTotal, on = ['companyName', 'id_cncf','x'], how='left')

In [ ]:
#pd.unique(appl_hor3['id_posiblegpa'])
#specify path for export
import os
path = r'D:\traderxpro\data\cncf_h.txt'

# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_h6.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)